In [ ]:
!pip install transformers==4.33.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 sentence_transformers==2.2.2 chromadb==0.4.12 pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.5/426.5 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 97.8 MB/s eta 

In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
import chromadb
from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import os
import pandas as pd
import numpy as np

In [ ]:
from huggingface_hub import login
hf_api_key = ""
login(hf_api_key)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model_id = 'meta-llama/Meta-Llama-3-8B-Instruct'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)
time_1 = time()
model = transformers.AutoModelForCausalLM.from_pretrained(model_id).to(device)
model = model.half()
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

cuda:0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Prepare model, tokenizer: 133.376 sec.


In [ ]:
time_1 = time()
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.5,
    device=device
)

time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 1.497 sec.


In [ ]:
llm = HuggingFacePipeline(pipeline=query_pipeline, verbose=False)

In [ ]:
file_path = 'XBRL Terminology.xlsx'
df_all = pd.read_excel(file_path)

In [ ]:
df = df_all[:500].copy()
df

,Term,Explanation,Unnamed: 2,Unnamed: 3
0,abstract,An attribute of an element to indicate that th...,NaN,NaN
1,attribute,"A property of an element such as its name, bal...",NaN,NaN
2,authoritative reference,Citations to specific authoritative accounting...,NaN,NaN
3,axis (pl. axes),An instance document contains facts; an axis d...,NaN,NaN
4,axis-default relationship,The dimensional relationship indicating that t...,NaN,NaN
...,...,...,...,...
495,"Entity Listing, Par Value Per Share",The par value per share of security quoted in ...,NaN,NaN
496,Entity Number of Employees,Number of persons employed by the Entity,NaN,NaN
497,Entity Public Float,State aggregate market value of voting and non...,NaN,NaN
498,Demutualization Cost and Expense,The costs and expenses incurred pertaining to ...,NaN,NaN


In [ ]:
from langchain.docstore.document import Document

terms = df['Term'].tolist()
explanations = df['Explanation'].tolist()
documents = [{"text": term + " - " + explanation} for term, explanation in zip(terms, explanations)]

texts = [doc["text"] for doc in documents]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

all_splits = []
for text in texts:
    splits = text_splitter.split_text(text)
    all_splits.extend(splits)

all_splits = [Document(page_content=str(split)) for split in all_splits]

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="report_db_term")
vectordb.persist()

In [ ]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [ ]:
df_title = pd.DataFrame(columns=['Term', 'Generated Explanation', "Correct Answer"])
excel_path = 'terminology_rag_results_llama_500.xlsx'
df_title.to_excel(excel_path, index=False)

In [ ]:
prompt = "You are an expert in the financial field with deep expertise in the eXtensible Business Reporting Language (XBRL) standard. Please provide detailed explanations of the following XBRL terms: "

In [ ]:
for i in range(len(df)):
    term = df['Term'].values[i]
    explanation = df['Explanation'].values[i]
    query = prompt + " " + term
    result = qa.run(query)
    new_row = pd.DataFrame({'Term': [term], "Generated Answer": [result], "Correct Answer": [explanation]})

    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        book = writer.book
        startrow = writer.sheets['Sheet1'].max_row
        new_row.to_excel(writer, index=False, header=False, startrow=startrow)
    print(f"{i}" + " saved")

print("finished")